In [ ]:
from datasets import load_from_disk, concatenate_datasets

ds_korean = load_from_disk("../dataset/분류 모델 용/korean")["test"].select(range(1))
ds_cloth = load_from_disk("../dataset/분류 모델 용/cloth")["test"].select(range(1))
ds_race_middle_long = load_from_disk("../dataset/분류 모델 용/race_middle_long")["test"].select(range(1))
ds_race_middle_short = load_from_disk("../dataset/분류 모델 용/race_middle_short")["test"].select(range(1))
ds_race_high_long = load_from_disk("../dataset/분류 모델 용/race_high_long")["test"].select(range(1))
ds_race_high_short = load_from_disk("../dataset/분류 모델 용/race_high_short")["test"].select(range(1))


/home/nclabterm1/refresh/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import sys
sys.path.append('..')

from huggingface_hub import login
import torch
import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig
import os

In [ ]:
from dotenv import load_dotenv

load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")
model_id = "./output/0"
base_model_id = "microsoft/Phi-4-mini-instruct"

torch_dtype = torch.bfloat16

login(token)

model_kwargs = dict(
    attn_implementation="eager",
    torch_dtype=torch_dtype, 
    device_map="auto", 
)

model_kwargs["quantization_config"] = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=model_kwargs['torch_dtype'],
    bnb_4bit_quant_storage=model_kwargs['torch_dtype'],
)

args = SFTConfig(
    output_dir="output/results",         
    packing=True,             
    gradient_accumulation_steps=4,    
    gradient_checkpointing=True,       
    optim="adamw_torch_fused",      
    logging_steps=10,                   
    save_strategy="epoch",
    fp16=True if torch_dtype == torch.float16 else False,  
    bf16=True if torch_dtype == torch.bfloat16 else False, 

    #eval_strategy="epoch",
    #per_device_eval_batch_size=6,

    lr_scheduler_type="constant",         
    push_to_hub=False,                     
    report_to="tensorboard",         
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": True,
        }
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)
base = AutoModelForCausalLM.from_pretrained(base_model_id, **model_kwargs)
model = PeftModel.from_pretrained(base, model_id)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.92s/it]


In [ ]:
import re
import torch
from transformers import StoppingCriteria, StoppingCriteriaList

class StopOnAnswerIs(StoppingCriteria):
    def __init__(self, tokenizer, pattern=r"answer\s*is\s*:?\s*$", lookback_tokens=64):
        super().__init__()
        self.tokenizer = tokenizer
        self.regex = re.compile(pattern, flags=re.IGNORECASE)
        self.lookback = lookback_tokens

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        tail = input_ids[0, -self.lookback:].tolist()
        text = self.tokenizer.decode(tail, skip_special_tokens=True)
        return bool(self.regex.search(text))


In [ ]:
import build_dataset as bd

device = next(model.parameters()).device 

def safe_letter_token_id(tokenizer, letter: str) -> int:
    vocab = tokenizer.get_vocab()
    letter = letter.upper()
    for tok in (f"▁{letter}", f" {letter}", letter):
        if tok in vocab:
            return vocab[tok]
    ids = tokenizer.encode(f" {letter}", add_special_tokens=False)
    if len(ids) == 1:
        return ids[0]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[{'role': 'system', 'content': "You are a helpful assistant.\nProvide reasoning only (no mention or hints about options).\nAt the very end, output exactly: 'answer is X' where X is one of A,B,C,D,E.\nThe phrase 'answer is' must appear once, in lowercase, and only on the final line.\nDo not reveal or imply the answer prior to that final line."}, {'role': 'user', 'content': '\n[QUESTION]\n다음 글의 요지로 가장 적절한 것은?\n\n[PASSAGE]\nPeople sometimes make downward social comparisons ―\ncomparing themselves to inferior or worse-off others ― to feel\nbetter about themselves. This is self-enhancement at work.\nBut what happens when the only available comparison target\nwe have is superior or better off than we are? Can\nself-enhancement motives still be served in such situations?\nYes, they can, as captured by the self-evaluation maintenance\nmodel. According to this theory, we shift between two\nprocesses ― reflection and comparison ― in a way that lets us\nmaintain favorable self-views. In areas tha

In [6]:
def extract_gold_letter(ex):
    return ex["content"]

In [ ]:
import re, json, datetime
from tqdm.auto import tqdm
import torch
from transformers import StoppingCriteria, StoppingCriteriaList

def evaluate_until_answer_is(
    ds,
    ds_name,
    bd,                    
    model,
    tokenizer,
    max_items=None,
    do_sample=False,       
    temperature=0.7,
    top_p=0.9,
    lookback_tokens=64,
    save_results=True,
):
    device = next(model.parameters()).device
    model.eval()

    stopper = StoppingCriteriaList([StopOnAnswerIs(tokenizer, lookback_tokens=lookback_tokens)])
    option_letters = ['A', 'B', 'C', 'D', 'E']
    option_token_ids = [safe_letter_token_id(tokenizer, L) for L in option_letters]

    N = len(ds) if max_items is None else min(max_items, len(ds))
    correct = 0
    evaluated = 0
    stopped_cnt = 0
    skipped_cnt = 0
    results = []

    pbar = tqdm(range(N))
    for i in pbar:

        item = ds[i]
        item = bd.create_conversation_for_test(item)
        msgs = item['messages'][:2]
        gold = extract_gold_letter(item['messages'][2])

        prompt_ids = tokenizer.apply_chat_template(
            msgs, add_generation_prompt=True, return_tensors="pt"
        ).to(device)

        gen_out = model.generate(
            prompt_ids,
            max_new_tokens=256,
            do_sample=do_sample,
            temperature=temperature if do_sample else None,
            top_p=top_p if do_sample else None,
            stopping_criteria=stopper,
            return_dict_in_generate=True,
            use_cache=True,
        )
        seq = gen_out.sequences 
        gen_text = tokenizer.decode(seq[0, prompt_ids.shape[1]:], skip_special_tokens=True)
        
        is_skipped = False

        if not re.search(r"answer\s*is\s*:?\s*$", gen_text.strip(), re.IGNORECASE):
            skipped_cnt += 1
            is_skipped = True
        
        if is_skipped == False:
            stopped_cnt += 1

        with torch.inference_mode():
            out2 = model(seq.to(device))
            next_logits = out2.logits[:, -1, :].squeeze(0) 

        option_logits = next_logits[option_token_ids]          
        option_logprobs = torch.log_softmax(option_logits, dim=0)  
        option_probs = torch.softmax(option_logits, dim=0)       

        probs = option_probs.detach().cpu().float().tolist()
        pred_idx = int(torch.argmax(option_probs).item())
        pred = option_letters[pred_idx]

        if is_skipped == False:
            evaluated += 1
            
        is_correct = (pred == gold)

        if is_skipped == False:
            correct += int(is_correct)

        results.append({
            "index": i,
            "generated_until_stop": gen_text,
            "pred": pred,
            "probs": {L: float(p) for L, p in zip(option_letters, probs)},
            "gold": gold,
            "is_correct": is_correct,
            "is_skipped": is_skipped
        })

        pbar.set_postfix(
            acc=f"{(correct/max(1,evaluated))*100:.2f}%",
            stopped=stopped_cnt,
            skipped=skipped_cnt
        )

    final = {
        "model_id": getattr(model.config, "name_or_path", "unknown"),
        "num_items_total": N,
        "num_evaluated": evaluated,
        "num_stopped": stopped_cnt,
        "num_skipped": skipped_cnt,
        "accuracy_no_skipped": f"{(correct/max(1,evaluated))*100:.2f}%",
        "results": results
    }

    if save_results:
        ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        path = f"./eval/results_{ds_name}_{ts}.json"
        os.makedirs(os.path.dirname(path), exist_ok=True)
        with open(path, "w", encoding="utf-8") as f:
            json.dump(final, f, ensure_ascii=False, indent=2)
        print(f"\nSaved: {path}")

    print(f"\nEvaluated {evaluated}/{N} (stopped={stopped_cnt}, skipped={skipped_cnt})")
    print(f"Accuracy: {final['accuracy_no_skipped']}")
    return final

In [8]:
ds = [ds_korean, ds_cloth, ds_race_middle_long, ds_race_middle_short, ds_race_high_long, ds_race_high_short]
ds_name = ["ds_korean", "ds_cloth", "ds_race_middle_long", "ds_race_middle_short", "ds_race_high_long", "ds_race_high_short"]

In [9]:
for i in range(6):
    evaluate_until_answer_is(
        ds[i],
        ds_name[i],
        bd,
        model,
        tokenizer,
        max_items=None,  
        do_sample=False,   
        save_results=True,
    )

100%|██████████| 124/124 [16:32<00:00,  8.00s/it, acc=77.78%, skipped=7, stopped=117]



Saved: ./eval/results_ds_korean_20250908_195643.json

Evaluated 117/124 (stopped=117, skipped=7)
Accuracy: 77.78%


100%|██████████| 2000/2000 [3:44:19<00:00,  6.73s/it, acc=66.60%, skipped=6, stopped=1994]  



Saved: ./eval/results_ds_cloth_20250908_234103.json

Evaluated 1994/2000 (stopped=1994, skipped=6)
Accuracy: 66.60%


100%|██████████| 772/772 [1:39:09<00:00,  7.71s/it, acc=69.48%, skipped=25, stopped=747]


...


Saved: ./eval/results_ds_race_middle_short_20250909_023526.json

Evaluated 648/664 (stopped=648, skipped=16)
Accuracy: 70.99%


100%|██████████| 1686/1686 [3:57:44<00:00,  8.46s/it, acc=61.61%, skipped=71, stopped=1615]  



Saved: ./eval/results_ds_race_high_long_20250909_063310.json

Evaluated 1615/1686 (stopped=1615, skipped=71)
Accuracy: 61.61%


100%|██████████| 1812/1812 [4:06:30<00:00,  8.16s/it, acc=67.62%, skipped=67, stopped=1745]  


Saved: ./eval/results_ds_race_high_short_20250909_103941.json

Evaluated 1745/1812 (stopped=1745, skipped=67)
Accuracy: 67.62%


In [ ]:
import json
from pathlib import Path

IN_DIR = Path("./eval")
OUT1 = Path("./eval_1")
OUT2 = Path("./eval_2")
OUT1.mkdir(parents=True, exist_ok=True)
OUT2.mkdir(parents=True, exist_ok=True)

def recalc_stats(data: dict, new_results: list) -> dict:
    d = dict(data)
    n_total = len(new_results)
    n_skipped = sum(1 for r in new_results if r.get("is_skipped", False))
    n_eval = n_total - n_skipped
    n_stopped = n_eval

    n_correct = sum(
        1 for r in new_results
        if not r.get("is_skipped", False) and r.get("is_correct", False)
    )
    acc = f"{(n_correct / n_eval) * 100:.2f}%" if n_eval > 0 else "0.00%"

    d["results"] = new_results
    d["num_items_total"] = n_total
    d["num_evaluated"] = n_eval
    d["num_stopped"] = n_stopped
    d["num_skipped"] = n_skipped
    d["accuracy_no_skipped"] = acc
    return d

def count_metrics(results: list):
    n_total = len(results)
    n_skipped = sum(1 for r in results if r.get("is_skipped", False))
    n_eval = n_total - n_skipped
    n_correct_excl_skip = sum(
        1 for r in results
        if not r.get("is_skipped", False) and r.get("is_correct", False)
    )
    acc_excl = (n_correct_excl_skip / n_eval * 100) if n_eval > 0 else 0.0
    acc_incl = (n_correct_excl_skip / n_total * 100) if n_total > 0 else 0.0
    return {
        "correct_excl": n_correct_excl_skip,
        "eval_cnt": n_eval,
        "total_cnt": n_total,
        "skipped": n_skipped,
        "acc_excl": acc_excl,
        "acc_incl": acc_incl,
    }

processed = []

agg_1 = {"correct_excl": 0, "eval_cnt": 0, "total_cnt": 0, "skipped": 0}
agg_2 = {"correct_excl": 0, "eval_cnt": 0, "total_cnt": 0, "skipped": 0}

for path in sorted(IN_DIR.glob("*.json")):
    with path.open("r", encoding="utf-8") as f:
        data = json.load(f)

    results = data.get("results", [])
    n = len(results)

    n_front = n // 2
    front = results[:n_front]
    back = results[n_front:] 

    data_front = recalc_stats(data, front)
    data_back = recalc_stats(data, back)

    out1 = OUT1 / path.name
    out2 = OUT2 / path.name

    with out1.open("w", encoding="utf-8") as f:
        json.dump(data_front, f, ensure_ascii=False, indent=2)
    with out2.open("w", encoding="utf-8") as f:
        json.dump(data_back, f, ensure_ascii=False, indent=2)

    m1 = count_metrics(front)
    m2 = count_metrics(back)

    processed.append((
        path.name,
        len(front), len(back),
        f"{m1['acc_excl']:.2f}%", f"{m2['acc_excl']:.2f}%",  
        f"{m1['acc_incl']:.2f}%", f"{m2['acc_incl']:.2f}%", 
    ))

    for k in agg_1: agg_1[k] += m1[k]
    for k in agg_2: agg_2[k] += m2[k]

for name, nf, nb, accf_ex, accb_ex, accf_in, accb_in in processed:
    print(
        f"{name}: "
        f"eval_1(front)={nf}개 acc_excl_skip={accf_ex} | acc_incl_skip={accf_in}  ||  "
        f"eval_2(back)={nb}개 acc_excl_skip={accb_ex} | acc_incl_skip={accb_in}"
    )

def pretty_overall(agg):
    acc_excl = (agg["correct_excl"] / agg["eval_cnt"] * 100) if agg["eval_cnt"] > 0 else 0.0
    acc_incl = (agg["correct_excl"] / agg["total_cnt"] * 100) if agg["total_cnt"] > 0 else 0.0
    return acc_excl, acc_incl

acc1_ex, acc1_in = pretty_overall(agg_1)
acc2_ex, acc2_in = pretty_overall(agg_2)

print(f"eval_1(front): "
      f"correct={agg_1['correct_excl']}, "
      f"eval={agg_1['eval_cnt']}, total={agg_1['total_cnt']}, skipped={agg_1['skipped']} -> "
      f"acc_excl_skip={acc1_ex:.2f}% | acc_incl_skip={acc1_in:.2f}%")
print(f"eval_2(back) : "
      f"correct={agg_2['correct_excl']}, "
      f"eval={agg_2['eval_cnt']}, total={agg_2['total_cnt']}, skipped={agg_2['skipped']} -> "
      f"acc_excl_skip={acc2_ex:.2f}% | acc_incl_skip={acc2_in:.2f}%")

def peek(dir_name: str, file_name: str):
    p = Path(dir_name) / file_name
    with p.open("r", encoding="utf-8") as f:
        d = json.load(f)
    head = {k: d[k] for k in ["model_id", "num_items_total", "num_evaluated", "num_stopped", "num_skipped", "accuracy_no_skipped"] if k in d}
    print(p, "->", head)
